# Cleaning data for TN_MVR_2018-2022 dataset


In [1]:
# Import necessary libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import time

from vininfo import Vin #get manufacturer
from pyvin import VIN #get model
from util.vehiclesModeler import *
from util.vin_api import *
from util.location import *

In [2]:
# Path to data folders
## Folder that contains the raw data
OLD_PATH="./data/oldData/"
## Folder that contains the processed data
PROC_PATH="./data/processedData/"

In [ ]:
# Import CSV data from datasets (tn_mvr_2018_2022)
df = pd.read_csv(f"{OLD_PATH}/tn_mvr_2018-2022.csv", sep="\t",  low_memory=False)

In [5]:
# Check for total values
print("Dataframe Total Values (Before Dropping Missing):", df.shape[0])

# Check the data for missing values
print("Dataframe Total Missing Values:", df.isna().sum().sum())
print("Dataframe Columns Missing Values:\n", df.isna().sum())

# Remove the missing values (drop rows with missing values)
df = df.dropna()

# Check for total values
print("Dataframe Total Values (After Dropping Missing):", df.shape[0])

Dataframe Total Values (Before Dropping Missing): 9831774
Dataframe Total Missing Values: 333030
Dataframe Columns Missing Values:
 VIN                            4
VehicleCost               331544
OdometerTypeCode               0
OdometerReading                1
CountyName                     0
ZIP5                           0
ModelYear                      0
MakeCode                       0
ModelCode                    586
VehicleTypeDescription         0
NewUsedCode                    0
TitleIssueDate               895
PurchaseDate                   0
dtype: int64
Dataframe Total Values (After Dropping Missing): 9498752


In [6]:
# Rename the dataframe columns
df.columns = ["vin", "price", "odometer_type","mileage", "county", "zip", "model_year",
              "make", "model", "vehicle_type", "new_used", "title_issue_date", "purchase_date"]

# Set the date types
df["title_issue_date"] = pd.to_datetime(df["title_issue_date"], format="%Y-%m-%d")
df["purchase_date"] = pd.to_datetime(df["purchase_date"], format="%Y-%m-%d", errors="coerce")

In [7]:
# Print out the patterns of the dataset
print("Dataframe Shape and Columns:")
print(df.shape)
print(df.columns)
print()

# Check initial values of the dataset
print("Dataframe initial values:")
print(df.sample(2))
print()

Dataframe Shape and Columns:
(9498752, 13)
Index(['vin', 'price', 'odometer_type', 'mileage', 'county', 'zip',
       'model_year', 'make', 'model', 'vehicle_type', 'new_used',
       'title_issue_date', 'purchase_date'],
      dtype='object')

Dataframe initial values:
                      vin     price  odometer_type  mileage      county  \
382766  1C4RJKDG7N8542387  63219.00              0     13.0  Montgomery   
378279  1C6JJTEG0ML600182  68311.23              0  16228.0      Shelby   

          zip  model_year  make model vehicle_type new_used title_issue_date  \
382766  37040        2022  JEEP   GCH         AUTO        N       2022-06-30   
378279  38118        2021  JEEP   GLA        TRUCK        U       2022-11-08   

       purchase_date  
382766    2022-06-03  
378279    2022-11-08  



In [8]:
# Get basic info of the dataframe
print("Dataframe Info:")
print(df.info(show_counts=False))
print()

Dataframe Info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9498752 entries, 0 to 9831773
Data columns (total 13 columns):
 #   Column            Dtype         
---  ------            -----         
 0   vin               object        
 1   price             float64       
 2   odometer_type     int64         
 3   mileage           float64       
 4   county            object        
 5   zip               object        
 6   model_year        int64         
 7   make              object        
 8   model             object        
 9   vehicle_type      object        
 10  new_used          object        
 11  title_issue_date  datetime64[ns]
 12  purchase_date     datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(2), object(7)
memory usage: 1014.6+ MB
None



In [9]:
# Describe the numeric columns
print("Numeric Columns:")
print(df.describe().round(decimals = 1))
print()

Numeric Columns:
              price  odometer_type       mileage  model_year
count  9.498752e+06      9498752.0  9.498752e+06   9498752.0
mean   2.163936e+06            0.5  3.845500e+04      2010.4
std    2.792655e+09            0.8  4.529582e+06        10.0
min   -7.000000e+00            0.0 -2.256996e+08      1005.0
25%    0.000000e+00            0.0  0.000000e+00      2005.0
50%    0.000000e+00            0.0  8.000000e+00      2013.0
75%    1.000000e+03            1.0  3.918600e+04      2018.0
max    8.118003e+12            9.0  2.147484e+09      2890.0



In [10]:
# Describe the non-numeric columns
print("Non-numeric Columns:")
print(df.describe(include = ["object","category"]))
print()

Non-numeric Columns:
                  vin    county      zip     make    model vehicle_type  \
count         9498752   9498752  9498752  9498752  9498752      9498752   
unique        7423553        97     5950     8568    24643           16   
top     CCZ264F146470  Davidson    37211     FORD      F15         AUTO   
freq               17    812796   112367  1318066   280660      6684615   

       new_used  
count   9498752  
unique        2  
top           U  
freq    7575502  



In [11]:
# Describe the dates
print("Date Columns:")
print(df.describe(include = ["datetime"], datetime_is_numeric=True))
print()

Date Columns:
                    title_issue_date                  purchase_date
count                        9498752                        9498181
mean   2020-07-01 05:41:17.225473536  2020-02-07 20:53:56.619558400
min              1971-06-12 00:00:00            1720-02-08 00:00:00
25%              2019-04-08 00:00:00            2018-12-28 00:00:00
50%              2020-07-21 00:00:00            2020-04-19 00:00:00
75%              2021-09-28 00:00:00            2021-07-14 00:00:00
max              2022-12-30 00:00:00            2220-03-07 00:00:00



In [12]:
df.head()

,vin,price,odometer_type,mileage,county,zip,model_year,make,model,vehicle_type,new_used,title_issue_date,purchase_date
0,\1FTRX07L53KD87737,0.0,1,0.0,State,37207,2003,FORD,F15,AUTO,U,2018-07-27,2018-06-04
1,0000000V464048305,0.0,1,0.0,State,37167,1995,MITS,PAJ,AUTO,U,2020-06-24,2020-05-13
2,0000161231,1800.0,1,0.0,State,37397,1955,CHEV,BEL,AUTO,U,2021-12-13,2021-12-08
3,0005400FKXX,0.0,1,0.0,Fentress,38553,1955,CHEV,310,AUTO,U,2019-03-07,2019-03-04
4,0096456,300.0,1,0.0,Blount,37804,1966,VOLK,BUG,AUTO,U,2022-11-22,2022-11-15


In [13]:
df[df.price!=0].shape

(3610711, 13)

In [14]:
df.make = df.make.apply(str.lower)

In [15]:
df.model = df.model.apply(str.lower)

In [16]:
df.vehicle_type.unique()

array(['AUTO', 'MOBILE HOME/HOUSE TR', 'FREIGHT/SEMI TRAILER',
       'UTILITY (BOX/CARGO) ', 'LIVESTOCK/HORSE TRAI', 'OTHER', 'TRUCK',
       'UTILITY (FLAT BED) T', 'MOTORCYCLE', 'CAMPER TRAILER',
       'WATERCRAFT TRAILER', 'BUS - COMMERCIAL', 'RECREATION/MOTOR HOM',
       'Multipurpose Vehicle', 'SCHOOL BUS', 'VAN'], dtype=object)

In [17]:
df = df[(df.vehicle_type=="AUTO") | \
        (df.vehicle_type=="TRUCK") | \
        (df.vehicle_type=="MOTORCYCLE") | \
        (df.vehicle_type=="BUS - COMMERCIAL") | \
        (df.vehicle_type=="Multipurpose Vehicle") | \
        (df.vehicle_type=="VAN") | \
        (df.vehicle_type=="SCHOOL BUS") | \
        (df.vehicle_type=="OTHER")]

In [18]:
df = df[df.price!=0]

In [19]:
df.shape

(3450442, 13)

In [20]:
def parse_vin_to_manufacturer(vin: str) -> str:
    try:
        make = Vin(vin).manufacturer
        if make == "UnsupportedBrand":
            return np.nan
        return make.lower()
    except:
        return np.nan

In [21]:
df.make = df.vin.apply(parse_vin_to_manufacturer)

In [22]:
df.shape

(3450442, 13)

In [23]:
df[df.duplicated()].shape

(0, 13)

In [24]:
df.dropna(inplace=True)

In [25]:
df.make = df.make.apply(filter_manufacturers)

In [26]:
df.isna().sum()

vin                     0
price                   0
odometer_type           0
mileage                 0
county                  0
zip                     0
model_year              0
make                20885
model                   0
vehicle_type            0
new_used                0
title_issue_date        0
purchase_date           0
dtype: int64

In [27]:
df.dropna(inplace=True)

In [28]:
df.model = list(map(
    lambda make, model: 
    get_parse_from_model_str(make).parse_model(model).get_model(), 
    df.make, 
    df.model
))

In [29]:
df.dropna(inplace=True)
df.shape

(537227, 13)

In [30]:
df["lat"] = df.zip.apply(get_lat)

In [31]:
df["long"] = df.zip.apply(get_long)

In [32]:
df["state"] = df.zip.apply(get_state)

In [33]:
df.isna().sum()

vin                    0
price                  0
odometer_type          0
mileage                0
county                 0
zip                    0
model_year             0
make                   0
model                  0
vehicle_type           0
new_used               0
title_issue_date       0
purchase_date          0
lat                 1441
long                1441
state               1441
dtype: int64

In [34]:
df.dropna(inplace=True)

In [35]:
df.shape

(535786, 16)

In [85]:
vehicles = pd.read_csv(f"{PROC_PATH}/vehicles.csv")[["manufacturer", "model", "fuel"]]

In [91]:
vehicles=vehicles.drop_duplicates()

In [93]:
vehicles.head()

,manufacturer,model,fuel
0,ford,f150,gas
1,gmc,sierra 2500,gas
2,chevrolet,c1500,gas
3,toyota,tacoma,gas
4,chevrolet,colorado,gas


In [80]:
unique_cars = df[["make", "model"]]

In [82]:
unique_cars.shape

(535786, 2)

In [95]:
unique_cars = unique_cars.drop_duplicates()

In [108]:
vehicles[(vehicles.manufacturer=="ford") & (vehicles.model=="f150")].fuel.iloc[0]

'gas'

In [112]:
for row in unique_cars.index:
    make = unique_cars.loc[row, "make"]
    model = unique_cars.loc[row, "model"]
    gas = vehicles[
        (vehicles.manufacturer==make) & (vehicles.model==model)
    ].fuel
    if type(gas)!=str:
        try:
            gas = gas.iloc[0]
        except:
            gas = "gas"
    unique_cars.loc[row] = {
        "make":make,
        "model":model,
        "fuel":gas
    }

In [114]:
unique_cars.fuel.unique()

array(['gas', 'diesel', 'electric', 'other', 'hybrid'], dtype=object)

In [115]:
def filter_fuel(fuel):
    if fuel in ("gas", "diesel", "electric"):
        return fuel
    return "gas"

In [116]:
unique_cars.fuel = unique_cars.fuel.apply(filter_fuel)

In [121]:
index = {}
for row in unique_cars.index:
    index[(unique_cars.loc[row, "make"], unique_cars.loc[row,"model"])] = row

In [122]:
df["fuel"] = list(
    map(
        lambda make, model:
            unique_cars.loc[ index[(make, model)] ].fuel,
        df.make,
        df.model
    )
)

In [5]:
(df.model_year==0).any()

False

In [10]:
df = df[df.price>0]

## Save resulting data

In [126]:
df.to_csv(f"{PROC_PATH}/tn_mvr.csv")